In [1]:
import sys
sys.path.append('/Users/shuyangdu/Desktop/ZillowChallenge/zillow-kaggle-challenge')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from data_process.column_schema import (PROPERTIES_RENAME_DICT, TRANSACTION_RENAME_DICT, 
                                        NUMERICAL_COLS, CATEGORICAL_COLS)
from data_process.data_process_pipeline import DataProcessPipeline
from data_process.data_transform_processor import DataProcessor
from lightgbm import LGBMRegressor
from models.tree_models.lgbm import LGBM
from sklearn.metrics import mean_absolute_error
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df_all = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged.csv')

# Data process

In [16]:
data_processor = DataProcessor()

In [17]:
X_all = DataProcessor.pre_process(df_all)
y_all = df_all['logerror'].values

In [18]:
iterator = data_processor.k_fold()

In [19]:
train_idx, test_idx = iterator.next()

In [20]:
X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val, y_val = X_all[test_idx], y_all[test_idx]

In [21]:
X_train = data_processor.fit_transform(X_train)

In [22]:
X_val = data_processor.transform(X_val)

# Construct Model

In [25]:
params = {
    'max_bin': 80,
    'learning_rate': 0.0116,
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'feature_fraction': 0.94,
    'bagging_fraction': 0.85,
    'bagging_freq': 80,
    'num_leaves': 110,
    'lambda_l2': 86.9,
    'n_estimators': 450,
}

In [26]:
model = LGBM(
#     feature_name=data_pipeline.original_feature_cols,
    categorical_feature=data_processor.categorical_col_idx,
    **params
)

In [27]:
model.fit(X_train, y_train)

In [23]:
model = LGBMRegressor(**params)

In [24]:
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1')

[1]	valid_0's l1: 0.0672402
[2]	valid_0's l1: 0.0672185
[3]	valid_0's l1: 0.0671978
[4]	valid_0's l1: 0.0671766
[5]	valid_0's l1: 0.0671602
[6]	valid_0's l1: 0.0671404
[7]	valid_0's l1: 0.0671218
[8]	valid_0's l1: 0.0671063
[9]	valid_0's l1: 0.0670897
[10]	valid_0's l1: 0.0670714
[11]	valid_0's l1: 0.0670546
[12]	valid_0's l1: 0.0670383
[13]	valid_0's l1: 0.0670219
[14]	valid_0's l1: 0.0670059
[15]	valid_0's l1: 0.0669906
[16]	valid_0's l1: 0.0669743
[17]	valid_0's l1: 0.0669593
[18]	valid_0's l1: 0.0669448
[19]	valid_0's l1: 0.0669309
[20]	valid_0's l1: 0.0669168
[21]	valid_0's l1: 0.0669044
[22]	valid_0's l1: 0.0668915
[23]	valid_0's l1: 0.0668777
[24]	valid_0's l1: 0.0668658
[25]	valid_0's l1: 0.0668528
[26]	valid_0's l1: 0.0668409
[27]	valid_0's l1: 0.06683
[28]	valid_0's l1: 0.0668195
[29]	valid_0's l1: 0.0668074
[30]	valid_0's l1: 0.0667947
[31]	valid_0's l1: 0.066784
[32]	valid_0's l1: 0.0667735
[33]	valid_0's l1: 0.0667617
[34]	valid_0's l1: 0.0667515
[35]	valid_0's l1: 0.06674

[283]	valid_0's l1: 0.0660475
[284]	valid_0's l1: 0.0660466
[285]	valid_0's l1: 0.0660451
[286]	valid_0's l1: 0.0660437
[287]	valid_0's l1: 0.0660432
[288]	valid_0's l1: 0.0660421
[289]	valid_0's l1: 0.0660413
[290]	valid_0's l1: 0.0660403
[291]	valid_0's l1: 0.0660401
[292]	valid_0's l1: 0.0660394
[293]	valid_0's l1: 0.0660384
[294]	valid_0's l1: 0.066037
[295]	valid_0's l1: 0.0660364
[296]	valid_0's l1: 0.0660347
[297]	valid_0's l1: 0.0660348
[298]	valid_0's l1: 0.0660336
[299]	valid_0's l1: 0.0660338
[300]	valid_0's l1: 0.0660328
[301]	valid_0's l1: 0.066032
[302]	valid_0's l1: 0.0660318
[303]	valid_0's l1: 0.0660314
[304]	valid_0's l1: 0.066031
[305]	valid_0's l1: 0.0660298
[306]	valid_0's l1: 0.0660298
[307]	valid_0's l1: 0.0660288
[308]	valid_0's l1: 0.0660287
[309]	valid_0's l1: 0.0660278
[310]	valid_0's l1: 0.0660267
[311]	valid_0's l1: 0.0660259
[312]	valid_0's l1: 0.066024
[313]	valid_0's l1: 0.0660231
[314]	valid_0's l1: 0.0660226
[315]	valid_0's l1: 0.066022
[316]	valid_0's

LGBMRegressor(bagging_fraction=0.85, bagging_freq=80, boosting_type='gbdt',
       colsample_bytree=1, feature_fraction=0.94, lambda_l2=86.9,
       learning_rate=0.0116, max_bin=80, max_depth=-1,
       min_child_samples=10, min_child_weight=5, min_split_gain=0,
       n_estimators=450, nthread=-1, num_leaves=110,
       objective='regression_l1', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1)

In [28]:
y_pred = model.predict(X_val)

In [20]:
np.unique(y_pred)

array([-0.27724017, -0.242604  , -0.22073611, ...,  0.18763668,
        0.22574773,  0.27805737])

In [13]:
mean_absolute_error(y_val, y_pred)

0.065969168600461162

In [29]:
mean_absolute_error(y_val, y_pred)

0.065955525236131671

# Deprecated code for old data process pipeline

In [3]:
data_pipeline = DataProcessPipeline(encode_mode='label')

In [4]:
df = data_pipeline.pre_process(df_all)

In [5]:
iterator = data_pipeline.k_fold(df, 52)

In [6]:
df_train, df_val = iterator.next()

In [7]:
df_train = data_pipeline.post_process(df_train, is_train=True)
df_val = data_pipeline.post_process(df_val, is_train=False)

In [8]:
X_train = df_train[data_pipeline.final_feature_cols].values
y_train = df_train[data_pipeline.label_col].values
X_val = df_val[data_pipeline.final_feature_cols].values
y_val = df_val[data_pipeline.label_col].values